In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('./data/Merged-Formatted-Indian-Names.csv')

In [3]:
data.head()

,name,gender
0,barjraj,m
1,ramdin verma,m
2,sharat chandran,m
3,birender mandal,m
4,amit,m


In [4]:

data.size

60970

In [5]:
data.columns

Index(['name', 'gender'], dtype='object')

In [6]:
data.dtypes

name      object
gender    object
dtype: object

In [7]:
data = data.dropna()

In [8]:
data = data.reset_index(drop=True)

In [9]:
data.size

60860

In [10]:
data[data['gender'] == 'm'].size

29642

In [11]:
data[data['gender'] == 'f'].size

30702

In [12]:
data[data['gender'] == 'u'].size

516

In [13]:
data_names = data

In [14]:
data_names.size

60860

## Unisex: 0; Female: 1; Male: 2

In [15]:
data_names['gender'].replace({'u': 0,'f': 1,'m': 2},inplace=True)

In [16]:
data_names['gender'].unique()

array([2, 1, 0], dtype=int64)

In [17]:
data_names.dtypes

name      object
gender     int64
dtype: object

In [18]:
name_data = data_names['name']
training_labels = data_names['gender']

In [19]:
name_data[name_data.isna()].size

0

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer

In [21]:
cv = CountVectorizer()
training_data = cv.fit_transform(name_data)

In [22]:
from sklearn.model_selection import train_test_split
train_set, test_set, train_labels, test_labels = train_test_split(training_data, training_labels, random_state =42, test_size= 0.33)

In [23]:
print('Training Set: ', train_set.shape[0])
print('Test Set: ', test_set.shape[0])
print('Training Labels: ', train_labels.shape[0])
print('Test Labels: ', test_labels.shape[0])

Training Set:  20388
Test Set:  10042
Training Labels:  20388
Test Labels:  10042


In [24]:
from sklearn.naive_bayes import MultinomialNB

In [25]:
clf = MultinomialNB()
clf.fit(train_set, train_labels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## Training accuracy

In [26]:
clf.score(train_set,train_labels)*100

96.39003335295271

## Test accuracy

In [27]:
clf.score(test_set,test_labels)*100

90.4600677155945

## Prediction on new data

In [28]:
sample_name = ["Rahul"]
vect = cv.transform(sample_name).toarray()

In [29]:
vect

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [30]:
clf.predict(vect)

array([2], dtype=int64)

In [31]:
def predictGender(a):
    test_name = [a]
    vector = cv.transform(test_name).toarray()
    if clf.predict(vector)[0] == 0:
        print("Unisex")
    elif clf.predict(vector)[0] == 1:
        print("Female")
    elif clf.predict(vector)[0] == 2:
        print("Male")
    else:
        print("Unknown")

In [32]:
predictGender("Soumya Dey")

Male


In [33]:
predictGender("Purnendu Paul")

Male


In [34]:
predictGender("Anik Ghoshal")

Female
